In [ ]:
# -*- coding: utf-8 -*-



!pip install datasets
!pip install transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from torch.utils.data import DataLoader, RandomSampler

#"""## Preprocessing Function """

import json
import sys
sys.stderr = open('temp.text', 'w')

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, BertForQuestionAnswering, BertTokenizer, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained('bert-large-cased')
model = AutoModelForQuestionAnswering.from_pretrained('bert-large-cased')

training_data = []
def preprocess_squad_dataset(dataset, tokenizer):
    examples = []
    
    for entity in dataset['data']:
        paras = entity['paragraphs']
      
        for in_entity in paras:
            question_objects = in_entity['qas']
            context = in_entity['context']
            for in_in_entity in question_objects:
                question = in_in_entity['question']
                answers = in_in_entity['answers']
                
              
                for ans in answers: 
                    ansrs = []
                    start_positions = []
                    end_positions = []
                    start_index = ans['answer_start']
                    answer = ans['text']
                    tokid = 0
                    for i in range(start_index+1):
                        if(context[i] == ' '):
                            tokid+=1
                    ansrs.append(answer)
                    # start_positions.append(tokid)
                    # end_positions.append(len(answer.split(' ')) + tokid -1)
                # Tokenize the input
                    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, max_length=16, truncation=True, padding='max_length', return_tensors="pt")

                    input_ids = inputs['input_ids']
                    token_type_ids = inputs['token_type_ids']
                    attention_mask = inputs['attention_mask']
                    start_position = tokid
                    end_position = start_position + len(answer.split()) - 1

                    examples.append({
                        'input_ids': input_ids,
                        'token_type_ids': token_type_ids,
                        'attention_mask': attention_mask,
                        'start_positions': torch.tensor([start_position]),
                        'end_positions': torch.tensor([end_position])
                    }) 
                    break
                    
            
    return examples

"""## Load dataset and preprocess it"""

f = open("/content/train-v2.0.json")
data = json.load(f)
preprocessed_squad_dataset = preprocess_squad_dataset(data, tokenizer)



"""## Train Dataset in batches"""

num_epochs = 20

num_training_steps = len(preprocessed_squad_dataset) * num_epochs
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

model.train()
sys.stderr = sys.__stderr__

print("started printin")
batch_size = 32

print("started printin")

for epoch in range(num_epochs):
    for i in range(0, len(preprocessed_squad_dataset), batch_size):
        batch = preprocessed_squad_dataset[i:i+batch_size]

        input_ids = torch.stack([b['input_ids'][0] for b in batch])
        token_type_ids = torch.stack([b['token_type_ids'][0] for b in batch])
        attention_mask = torch.stack([b['attention_mask'][0] for b in batch])
        start_positions = torch.stack([b['start_positions'][0] for b in batch])
        end_positions = torch.stack([b['end_positions'][0] for b in batch])

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    print(epoch)

"""## Save the model"""

torch.save(model.state_dict(), 'my_model_weights.pt')

"""## Test on batches"""

import json 
batch_size = 32
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def predict_answer(questions, contexts, model, tokenizer):
    inputs = [question + tokenizer.sep_token + answer for question, answer in zip(questions, contexts)]

    inputs = tokenizer.batch_encode_plus(inputs, add_special_tokens=True, max_length=16, truncation=True, padding='max_length', return_tensors="pt")
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention_mask = inputs['attention_mask']

    outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

    # Convert start_scores and end_scores to tensors
    start_scores = outputs.start_logits.squeeze()
    end_scores = outputs.end_logits.squeeze()


    answers = []
    for i in range(len(questions)):
        # Get the most likely beginning and end of the answer
        answer_start = torch.argmax(start_scores[i])
        answer_end = torch.argmax(end_scores[i]) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[i][answer_start:answer_end]))
        answer = answer.replace('[CLS]', '').replace('[SEP]', '').strip()
        answers.append(answer)

    return answers

with open('/content/dev-v2.0.json', 'r') as f:
    test_data = json.load(f)

predictions = {}
batch_questions = []
batch_contexts = []
batch_ids = []
for item in test_data['data']:
    for para in item['paragraphs']:
        context = para['context']
        for question in para['qas']:
            question_text = question['question']
            question_id = question['id']
            batch_questions.append(question_text)
            batch_contexts.append(context)
            batch_ids.append(question_id)
            if len(batch_questions) == batch_size:
                batch_answers = predict_answer(batch_questions, batch_contexts, model, tokenizer)
                for i in range(len(batch_questions)):
                     import json
batch_size = 32
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def predict_answer(questions, contexts, model, tokenizer):
    inputs = [question + tokenizer.sep_token + answer for question, answer in zip(questions, contexts)]

    inputs = tokenizer.batch_encode_plus(inputs, add_special_tokens=True, max_length=16, truncation=True, padding='max_length', return_tensors="pt")
    input_ids = inputs['input_ids']
    token_type_ids = ['i…e_ids']
    
        for question in para['qas']:
            question_text = question['question']
            question_id = question['id']
            batch_questions.append(question_text)
            batch_contexts.append(context)
            batch_ids.append(question_id)
            if len(batch_questions) == batch_size:
                batch_answers = predict_answer(batch_questions, batch_contexts, model, tokenizer)
                for i in range(len(batch_questions)):
                    predictions[batch_ids[i]] = batch_answers[i]
                batch_questions = []
                batch_contexts = []
                batch_ids = []

if len(batch_questions) > 0:
    batch_answers = predict_answer(batch_questions, batch_contexts, model, tokenizer)
    for i in range(len(batch_questions)):
        predictions[batch_ids[i]] = batch_answers[i]

with open('predictions1.json', 'w') as f:
    json.dump(predictions, f)